In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv,eig, det
from math import sqrt,exp,pi,log

import pickle
from joblib import dump, load


from scipy.stats import chi2_contingency
from sklearn import utils
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import matplotlib as mpl
import matplotlib.pyplot as plt

# read data

In [4]:
data = pd.read_excel('list-GP.xlsx',header=0)
# data = pd.read_excel('list-RFE.xlsx',header=0)
data = data.iloc[:,1:]
data

,GH,A,B,C,D,E
0,0.477963,1.335427,1.107740,1.632211,1.324698,0.113525
1,1.358324,0.024944,0.089729,-0.193301,-0.092795,-0.479850
2,0.696354,1.176630,1.744841,2.391572,2.053517,-0.642836
3,0.567620,2.334843,2.693940,2.195762,2.485904,1.118101
4,1.239236,0.645821,0.492072,0.013845,0.048179,-1.158008
...,...,...,...,...,...,...
145,-0.797232,4.708087,5.037075,4.243622,5.163987,3.178900
146,-0.918804,5.702825,5.145200,5.706707,3.910995,4.074553
147,-3.566696,13.284310,11.825719,8.187209,8.222029,7.073138
148,-0.190484,6.251279,6.008799,6.278541,6.277266,5.089915


# initial parameter setting

In [14]:
global N
global test_size     
N=20     # runing time for prediction
test_size=0.20   # the ratio of testing set

# Defined function

In [15]:
##  Model regression
def fitRegression(inputData,outputData,test_size,N,search):
    r2 = []
    rmse = []   # 存储算出的r2和rmse
    imp = []
    for i in range(N):
        print('---------round: %d---------'%i)
        X_train,X_test,y_train,y_test = train_test_split(inputData,outputData,test_size=test_size,random_state=None) # 随机分割数据 
                                                                                                                 # 该方法对dataframe也适用
        #分别初始化对特征和目标值的标准化器
        min_max_scaler = preprocessing.MinMaxScaler()
        #分别对训练和测试数据的特征以及目标值进行标准化处理
        X_train=min_max_scaler.fit_transform(X_train)
        X_test=min_max_scaler.fit_transform(X_test)
        y_train=min_max_scaler.fit_transform(y_train.reshape(-1,1)).ravel()#reshape(-1,1)指将它转化为1列，行自动确定
        y_test=min_max_scaler.fit_transform(y_test.reshape(-1,1)).ravel()#reshape(-1,1)指将它转化为1列，行自动确定        
        
        
        search.fit(X_train,y_train)      # 拟合
        regr = search.best_estimator_
        
        score = regr.score(X_test,y_test)  # 评分
        error = mean_squared_error(y_test,regr.predict(X_test),squared=False)
        importance = regr.feature_importances_

        r2.append(score)   #将评分存储至列表中
        rmse.append(error)
        imp.append(importance)
        print('r2:   %.3f'%score)
        print('RMSE: %.3f'%error)

        if i == 0:
            max_score = score           # 第0次运行时，记录此时的r2和模型
            choose_regr = pickle.dumps(regr)     # 模型将被保存至内存
            print('model has benn saved')
        if i>0:
            if score>max_score:
                max_score = score       # 在之后的运行中，更新最高r2的值，并保存模型以及此次随机分割后的数据
                X_train_best = X_train
                y_train_best = y_train
                X_test_best = X_test
                y_test_best = y_test
                choose_regr = pickle.dumps(regr)     
                print('model has benn saved')
        print('--------------------------')
    return r2,rmse,imp,X_train_best,y_train_best,X_test_best,y_test_best,choose_regr

In [16]:
##  GridCv range generation
def generateRfrParamGrid(low1,high1,num1,low2,high2,num2):      # 
    param_grid = {
        'n_estimators':np.arange(int(low1),int(high1),int(num1)),
        'max_depth':np.arange(int(low2),int(high2),int(num2))
    }
    return param_grid

# run rfr model

In [18]:
x_data = np.array(data.iloc[:,1:])
y_data = np.array(data.iloc[:,0])
param_grid = generateRfrParamGrid(300,500,10,3,7,1)
# param_grid = generateRfrParamGrid(300,400,100,3,7,3)
rfr = RandomForestRegressor()
search = GridSearchCV(rfr,param_grid=param_grid,n_jobs=-1,cv=10)
r2,rmse,imp,X_train_best,y_train_best,X_test_best,y_test_best,choose_rfr= \
fitRegression(x_data,y_data,test_size,N,search)

---------round: 0---------
r2:   0.722
RMSE: 0.127
model has benn saved
--------------------------
---------round: 1---------
r2:   0.760
RMSE: 0.130
model has benn saved
--------------------------
---------round: 2---------
r2:   0.848
RMSE: 0.090
model has benn saved
--------------------------
---------round: 3---------
r2:   0.747
RMSE: 0.127
--------------------------
---------round: 4---------
r2:   0.771
RMSE: 0.107
--------------------------
---------round: 5---------
r2:   0.741
RMSE: 0.136
--------------------------
---------round: 6---------
r2:   0.698
RMSE: 0.157
--------------------------
---------round: 7---------
r2:   0.739
RMSE: 0.100
--------------------------
---------round: 8---------
r2:   0.801
RMSE: 0.110
--------------------------
---------round: 9---------
r2:   0.651
RMSE: 0.122
--------------------------
---------round: 10---------
r2:   0.711
RMSE: 0.111
--------------------------
---------round: 11---------
r2:   0.676
RMSE: 0.143
--------------------------

# calculate feature importance

In [21]:
print(np.mean(r2))
print(np.mean(rmse))
imp = np.array(imp)
print(imp.mean(axis=0)) #特征值重要度
print(np.std(imp, axis=0,ddof = 1)) #特征值无偏std

0.7413139865531234
0.12504736298318933
[0.22380352 0.36332236 0.13152507 0.06542661 0.21592244]
[0.07917424 0.11112728 0.04250288 0.01142203 0.0897893 ]


# output feature importance

In [22]:
Feature_imp_std = np.vstack((imp.mean(axis=0),np.std(imp, axis=0,ddof = 1)))
Feature_imp_std = pd.DataFrame(Feature_imp_std)
# Feature_imp_std.to_csv("list-RFE-feature-importance.csv",index=False,sep=',')
Feature_imp_std.to_csv("list-GP-feature-importance.csv",index=False,sep=',')